Preprocessing Data:

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
from datetime import datetime, timedelta

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .config("spark.executor.memory", "15g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .appName('prprocess') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/09/04 12:17:11 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/04 12:17:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/04 12:17:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Consumer Details Dataframe Preprocess

In [3]:
consumer_details = spark.read.parquet(".././data/tables/consumer_user_details.parquet")
consumer_details.show()

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|      1|    1195503|
|      2|     179208|
|      3|    1194530|
|      4|     154128|
|      5|     712975|
|      6|     407340|
|      7|     511685|
|      8|     448088|
|      9|     650435|
|     10|    1058499|
|     11|     428325|
|     12|    1494640|
|     13|    1146717|
|     14|    1343547|
|     15|    1463076|
|     16|    1356405|
|     17|    1331093|
|     18|      80965|
|     19|    1226530|
|     20|    1390367|
+-------+-----------+
only showing top 20 rows



In [4]:
consumer_details.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



Change user_id and consumer_id to string type 

In [5]:
consumer_details = consumer_details.withColumn('user_id', F.col('user_id').cast('string'))
consumer_details = consumer_details.withColumn('consumer_id', F.col('consumer_id').cast('string'))

In [6]:
consumer_details.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- consumer_id: string (nullable = true)



Look for Dupilcate Values

In [7]:
consumer_details.count()

499999

In [8]:
#consumer_details.dropDuplicates().count()
consumer_details.select('user_id').distinct().count()

499999

In [9]:
consumer_details.select('consumer_id').distinct().count()

499999

No duplicate values have been found

Check for Missing Values 

In [10]:
consumer_details.filter(F.isnull('consumer_id')).count()


0

In [11]:
consumer_details.filter(F.isnan('consumer_id')).count()

0

In [12]:
consumer_details.filter(F.isnull('user_id')).count()

0

In [13]:
consumer_details.filter(F.isnan('user_id')).count()

0

In [15]:
#consumer_details.write.mode('overwrite').parquet('.././data/curated/consumer_details')

In [14]:
consumer_fraud = spark.read.option("header", True).csv(".././data/tables/consumer_fraud_probability.csv")
consumer_fraud.show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-25|92.73262811161372|
+-------+--

In [15]:
consumer_fraud.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



In [16]:
consumer_fraud = consumer_fraud.withColumn('order_datetime', F.col('order_datetime').cast('date'))

In [17]:
consumer_fraud = consumer_fraud.withColumn('fraud_probability', F.col('fraud_probability').cast('double'))

In [18]:
consumer_fraud.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- fraud_probability: double (nullable = true)



Check the range for fraude probability

In [19]:
consumer_fraud.filter(F.col('fraud_probability') < 0).show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
+-------+--------------+-----------------+



In [20]:
consumer_fraud.filter(F.col('fraud_probability') > 100).show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
+-------+--------------+-----------------+



The ranges of fraud probability lies in the correct range

Ensure all user_id's are properly referenced 

In [21]:
userids = consumer_details.select(F.col('user_id')).toPandas()['user_id']

In [22]:
userids = list(userids)
userids


['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13

In [23]:
from pyspark.sql.types import BooleanType
@F.udf(BooleanType())
def userID(x):
    if (x not in userids):
        return False
    else:
        return True
    
    

In [24]:
referenced = consumer_fraud.withColumn('Referenced',userID( F.col('user_id')))

In [25]:
referenced.show()

+-------+--------------+-----------------+----------+
|user_id|order_datetime|fraud_probability|Referenced|
+-------+--------------+-----------------+----------+
|   6228|    2021-12-19| 97.6298077657765|      true|
|  21419|    2021-12-10|99.24738020302328|      true|
|   5606|    2021-10-17|84.05825045251777|      true|
|   3101|    2021-04-17|91.42192091901347|      true|
|  22239|    2021-10-19|94.70342477508035|      true|
|  16556|    2022-02-20|89.65663294494827|      true|
|  10278|    2021-09-28|83.59136689427714|      true|
|  15790|    2021-12-30|71.77065889280253|      true|
|   5233|    2021-08-29|85.87123303878818|      true|
|    230|    2021-08-28|86.28328808934151|      true|
|  13601|    2021-12-26|83.13696487489679|      true|
|   6383|    2021-09-15| 66.2676451623754|      true|
|   3513|    2022-02-27|75.16981192247916|      true|
|  18658|    2021-10-19|82.98609082999361|      true|
|   5965|    2021-11-14|69.37164467869053|      true|
|  18714|    2021-11-14|83.7

In [26]:
referenced.filter(F.col('Referenced') == False).show()

+-------+--------------+-----------------+----------+
|user_id|order_datetime|fraud_probability|Referenced|
+-------+--------------+-----------------+----------+
+-------+--------------+-----------------+----------+



In [27]:
consumer_fraud.show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-25|92.73262811161372|
+-------+--

In [28]:
#consumer_fraud.write.mode('overwrite').parquet('.././data/curated/consumer_fraud')

In [29]:
transactions = spark.read.parquet(".././data/transactions/total_transactions")

In [30]:
transactions.show()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|
|  14936| 89518629617|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|
|  14936| 31101120643|25.228114942417797|8e301c0f-06ab-45c...|    2021-11-26|
|      2| 67978471888| 691.5028234458998|0380e9ad-b0e8-420...|    2021-11-26|
|  14936| 60956456424|102.13952056640888|5ac3da9c-5147-452...|    2021-11-26|
|      2| 47644196714| 644.5220654863093|4e368e44-86f8-4de...|    2021-11-26|
|  14938| 39649557865|209.12780951421405|4d78cd01-4bab-494...|    2021-11-26|
|      3| 88402174457| 141.0387993699113|c50c957d-ecfc-430...|  

In [31]:
tbl_merchants = spark.read.parquet(".././data/tables/tbl_merchants.parquet")
tbl_merchants.show(truncate=False)

+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|name                                |tags                                                                                                             |merchant_abn|
+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|Felis Limited                       |((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))|10023283211 |
|Arcu Ac Orci Corporation            |([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])                        |10142254217 |
|Nunc Sed Company                    |([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.40])                                          |10165489824 |
|Ult

In [42]:
tbl_merchants.printSchema()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: string (nullable = true)



In [41]:
tbl_merchants = tbl_merchants.withColumn('merchant_abn', F.col('merchant_abn').cast('string'))


In [43]:
tbl_merchants.select('merchant_abn').distinct().count()

4026

In [44]:
tbl_merchants.count()

4026

In [45]:
tbl_merchants.select('name').distinct().count()

4026

In [46]:
tbl_merchants.filter(F.isnull('name')).count()

0

In [47]:
tbl_merchants.filter(F.isnan('name')).count()

0

In [48]:
tbl_merchants.filter(F.isnan('merchant_abn')).count()

0

In [49]:
tbl_merchants.filter(F.isnull('merchant_abn')).count()

0

In [50]:
transactions.show()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|
|  14936| 89518629617|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|
|  14936| 31101120643|25.228114942417797|8e301c0f-06ab-45c...|    2021-11-26|
|      2| 67978471888| 691.5028234458998|0380e9ad-b0e8-420...|    2021-11-26|
|  14936| 60956456424|102.13952056640888|5ac3da9c-5147-452...|    2021-11-26|
|      2| 47644196714| 644.5220654863093|4e368e44-86f8-4de...|    2021-11-26|
|  14938| 39649557865|209.12780951421405|4d78cd01-4bab-494...|    2021-11-26|
|      3| 88402174457| 141.0387993699113|c50c957d-ecfc-430...|  

In [53]:
transactions.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [52]:
transactions = transactions.withColumn('merchant_abn', F.col('merchant_abn').cast('string'))
transactions = transactions.withColumn('user_id', F.col('user_id').cast('string'))

In [54]:
transactions.count()

14195505